In [1]:
import time
import random
import sqlite3
import numpy as np
import pandas as pd
from tqdm import tqdm
from datetime import datetime
from keras import layers, models
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service


In [2]:
# This opens a connection to two SQL databases.
schedule_connection = sqlite3.connect('nba-schedule.db')
schedule_cursor = schedule_connection.cursor()

player_connection = sqlite3.connect('nba-players.db')
player_cursor = player_connection.cursor()


In [3]:
# This block gets me today's date, and the date of our most recent database entry.

current_year = int(datetime.today().strftime('%Y'))
current_month = int(datetime.today().strftime('%m'))
current_day = int(datetime.today().strftime('%d')) 
if current_month > 9:
    current_season = current_year + 1
else:
    current_season = current_year
    
schedule_cursor.execute("SELECT year, month, day FROM games ORDER BY rowid DESC LIMIT 1")
most_recent = schedule_cursor.fetchall()
most_recent_year = most_recent[0][0]
most_recent_month = most_recent[0][1] 
most_recent_day = most_recent[0][2] 
if most_recent_month > 9:
    recent_season = most_recent_year + 1
else:
    recent_season = most_recent_year

In [4]:
options = webdriver.ChromeOptions()
options.add_argument("--incognito")
options.add_argument("--disable-blink-features")
options.add_argument("--disable-blink-features=AutomationControlled")
s = Service('/Users/AZC4063/Desktop/chromedriver2/chromedriver')
driver = webdriver.Chrome(options=options, service=s)

In [5]:
# This block is adding the first few datapoints to schedule/games.

months = ['january', 'february', 'march', 'april', 'may', 'june', 'july', 'august', 'september', 'october', 'november', 'december']
reference_month = most_recent_month
reference_day = most_recent_day

while recent_season < current_season or (recent_season == current_season and reference_month <= current_month):    
    print("The reference month is: " + str(reference_month) + " and the current month is: " + str(current_month))
    print("The recent season is: " + str(recent_season) + " and the current season is: " + str(current_season))
    month = months[reference_month-1]
    try:
        reference_url = "https://www.sports-website.com/leagues/NBA_" + str(recent_season) + "_games-" + month + ".html"
        driver.get(reference_url)
        time.sleep(4)
        table = driver.find_element(By.ID, "schedule").find_element(By.TAG_NAME, "tbody")
        games = table.find_elements(By.TAG_NAME, "tr")
        print(str(len(games)) + " games found.")
        for game in games:
            try:
                date = game.find_element(By.TAG_NAME, "th").text
                visitor_name = game.find_elements(By.TAG_NAME, "td")[1].text
                visitor_points = game.find_elements(By.TAG_NAME, "td")[2].text
                home_name = game.find_elements(By.TAG_NAME, "td")[3].text
                home_points = game.find_elements(By.TAG_NAME, "td")[4].text
                link = game.find_elements(By.TAG_NAME, "td")[5].find_element(By.TAG_NAME, "a").get_attribute('href')
                new_line = [(date), (visitor_name), (visitor_points), (home_name), (home_points), (link)]
                
                # This is testing to see if the entry already exists. 
                print("checking to see if this game already exists.")
                link_test = [link]
                schedule_cursor.execute("SELECT link FROM games WHERE link = (?)", link_test)
                link_test = schedule_cursor.fetchall()
                
                if len(link_test) == 0:
                    print("A new entry can be added.")
                    schedule_cursor.execute("""INSERT INTO games (date, visitor_name, visitor_points, home_name, home_points, link)
                                            VALUES (?,?,?,?,?,?)""", new_line)
                    schedule_connection.commit()
                else:
                    print("This entry already exists.")
            except Exception as e:
                print("Bad line, or: ")
    
    except Exception as e:
        print("No games this month or: ")

    if reference_month == 8:
        recent_season += 1
        reference_month += 1
    elif reference_month == 12:
        reference_month = 1            
    else:
        reference_month += 1             
    

    

The reference month is: 4 and the current month is: 4
The recent season is: 2023 and the current season is: 2023
71 games found.
checking to see if this game already exists.
This entry already exists.
checking to see if this game already exists.
This entry already exists.
checking to see if this game already exists.
This entry already exists.
checking to see if this game already exists.
This entry already exists.
checking to see if this game already exists.
This entry already exists.
checking to see if this game already exists.
This entry already exists.
checking to see if this game already exists.
This entry already exists.
checking to see if this game already exists.
This entry already exists.
checking to see if this game already exists.
This entry already exists.
checking to see if this game already exists.
This entry already exists.
checking to see if this game already exists.
This entry already exists.
checking to see if this game already exists.
This entry already exists.
checkin

In [7]:
# Mostly this will update the coach info where missning. 

coach_test = 0
while coach_test ==0:
    # This is finding the last entry in the database with an empty value for home_coach.
    schedule_cursor.execute("SELECT rowid, * FROM games WHERE home_coach IS NULL ORDER BY rowid DESC LIMIT 1")
    new_game = schedule_cursor.fetchall()
    if len(new_game) == 0:
        print("Home coaches complete.")
        coach_test = 1
    else:
        # This looks up the team name in the dictionary.
        team_name = [new_game[0][4]]
        player_cursor.execute("SELECT code FROM dictionary WHERE team_name = (?)", team_name)
        team_code = player_cursor.fetchall()
        team_url = "https://www.sports-website.com/teams/" + team_code[0][0] + "/" + str(current_season) + ".html"
        driver.get(team_url)
        time.sleep(4)
        coach_name = driver.find_element(By.XPATH, "//strong[text()='Coach:']/following-sibling::a").text
        print(coach_name)

        # This finds the last game entry we have for this team:
        schedule_cursor.execute("""SELECT rowid, home_coach FROM games WHERE home_name = (?) AND home_coach IS NOT NULL
                                ORDER BY rowid DESC LIMIT 1""", team_name)
        last_recorded_game = schedule_cursor.fetchall()
        last_recorded_coach = last_recorded_game[0][1]
        print(last_recorded_coach)

        if last_recorded_coach == coach_name:
            schedule_cursor.execute("SELECT rowid FROM games WHERE home_name = (?) AND home_coach IS NULL", team_name)
            blank_games = schedule_cursor.fetchall()
            for game in blank_games:
                data_input = [coach_name, game[0]]
                schedule_cursor.execute("UPDATE games SET home_coach = (?) WHERE rowid = (?)", data_input)  

            schedule_connection.commit()

        else:
            print("WARNING - THE COACH HAS CHANGED")
            print("Team name: " + str(team_name))
            print("Old coach: " + str(last_recorded_coach))
            print("New coach: " + str(coach_name))
            input("Press enter to continue.")

# We then repeat the code for the away team.
coach_test = 0
while coach_test ==0:
    # This is finding the last entry in the database with an empty value for visitor_coach.
    schedule_cursor.execute("SELECT rowid, * FROM games WHERE visitor_coach IS NULL ORDER BY rowid DESC LIMIT 1")
    new_game = schedule_cursor.fetchall()
    if len(new_game) == 0:
        print("Away coaches complete.")
        coach_test = 1
    else:
        # This looks up the team name in the dictionary.
        team_name = [new_game[0][2]]
        player_cursor.execute("SELECT code FROM dictionary WHERE team_name = (?)", team_name)
        team_code = player_cursor.fetchall()
        team_url = "https://www.sports-website.com/teams/" + team_code[0][0] + "/" + str(current_season) + ".html"
        driver.get(team_url)
        time.sleep(4)
        coach_name = driver.find_element(By.XPATH, "//strong[text()='Coach:']/following-sibling::a").text
        print(coach_name)

        # This finds the last game entry we have for this team:
        schedule_cursor.execute("""SELECT rowid, visitor_coach FROM games WHERE visitor_name = (?) AND visitor_coach IS NOT NULL
                                ORDER BY rowid DESC LIMIT 1""", team_name)
        last_recorded_game = schedule_cursor.fetchall()
        last_recorded_coach = last_recorded_game[0][1]
        print(last_recorded_coach)

        if last_recorded_coach == coach_name:
            schedule_cursor.execute("SELECT rowid FROM games WHERE visitor_name = (?) AND visitor_coach IS NULL", team_name)
            blank_games = schedule_cursor.fetchall()
            for game in blank_games:
                data_input = [coach_name, game[0]]
                schedule_cursor.execute("UPDATE games SET visitor_coach = (?) WHERE rowid = (?)", data_input)  

            schedule_connection.commit()

        else:
            print("WARNING - THE COACH HAS CHANGED")
            print("Team name: " + str(team_name))
            print("Old coach: " + str(last_recorded_coach))
            print("New coach: " + str(coach_name))
            input("Press enter to continue.")


Doc Rivers
Doc Rivers
Jamahl Mosley
Jamahl Mosley
Home coaches complete.
Michael Malone
Michael Malone
Mark Daigneault
Mark Daigneault
Chauncey Billups
Chauncey Billups
Erik Spoelstra
Erik Spoelstra
J.B. Bickerstaff
J.B. Bickerstaff
Away coaches complete.


In [8]:
# Next we're just adding the date info.
schedule_cursor.execute("SELECT rowid, date FROM games WHERE day IS NULL")
new_dates = schedule_cursor.fetchall()
months_list = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
for date in new_dates:
    chunks = date[1].split(", ")
    day = chunks[1].split(" ")[1]
    month = months_list.index(chunks[1].split(" ")[0])+1
    year = chunks[2]
    data_input = [day, month, year, date[0]]
    schedule_cursor.execute("UPDATE games SET (day, month, year) = (?, ?, ?) WHERE rowid = (?)", data_input)    
    schedule_connection.commit()
    

In [9]:
# This is updating the player names by checking the scorebox link.
schedule_cursor.execute("SELECT rowid, * FROM games WHERE visitor_player_1 IS NULL")
games = schedule_cursor.fetchall()

for game in games:
    driver.get(game[6])
    time.sleep(4)

    visitor_tag = visitor_box = driver.find_element(By.ID, "four_factors").find_elements(By.TAG_NAME, "a")[0].text
    home_tag = visitor_box = driver.find_element(By.ID, "four_factors").find_elements(By.TAG_NAME, "a")[1].text

    visitor_box = driver.find_element(By.ID, "box-"+visitor_tag+"-game-basic")
    home_box = driver.find_element(By.ID, "box-"+home_tag+"-game-basic")

    visitor_rows = visitor_box.find_elements(By.TAG_NAME, 'a')
    home_rows = home_box.find_elements(By.TAG_NAME, 'a')

    visitor_name_list = []
    home_name_list = []
    i = 1
    while i < 16:
        try:
            visitor_name = visitor_rows[i-1].get_attribute('href').split('.com')[1]
            visitor_name_list.append(visitor_name)
        except:
            visitor_name_list.append('')
        try:
            home_name = home_rows[i-1].get_attribute('href').split('.com')[1]
            home_name_list.append(home_name)
        except:
            home_name_list.append('')
        i += 1

    visitor_name_list.append(game[0])
    home_name_list.append(game[0])

    schedule_cursor.execute("""
        UPDATE games SET (
            home_player_1,
            home_player_2,
            home_player_3,
            home_player_4,
            home_player_5,
            home_player_6,
            home_player_7,
            home_player_8,
            home_player_9,
            home_player_10,
            home_player_11,
            home_player_12,
            home_player_13,
            home_player_14,
            home_player_15
        )= (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)
        WHERE rowid = (?)
        """, home_name_list)

    schedule_cursor.execute("""
        UPDATE games SET (
            visitor_player_1,
            visitor_player_2,
            visitor_player_3,
            visitor_player_4,
            visitor_player_5,
            visitor_player_6,
            visitor_player_7,
            visitor_player_8,
            visitor_player_9,
            visitor_player_10,
            visitor_player_11,
            visitor_player_12,
            visitor_player_13,
            visitor_player_14,
            visitor_player_15
        )= (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)
        WHERE rowid = (?)
        """, visitor_name_list)
    print("Completed row id: " + str(game[0]))
    schedule_connection.commit()

Completed row id: 17544
Completed row id: 17545
Completed row id: 17546
Completed row id: 17547
Completed row id: 17548


In [10]:
# This function will get a particular player's stats for a particular game, and add it to the players DB.
def game_scanner(game_list, player_name, playoff_reg): 
    player_cursor.execute("SELECT date FROM players WHERE name = (?)", [player_name])
    existing_dates = []
    date_list = player_cursor.fetchall()
    for date in date_list:
        existing_dates.append(date[0])
    
    metas = driver.find_element(By.ID, "meta").find_elements(By.TAG_NAME, "p")
    player_position = "no position found"
    for meta in metas:
        try:
            position_line = meta.text.split(': ')[0]
            if position_line == "Position":
                player_position = meta.text.split(': ')[1].split(' ▪ ')[0]
                print("Player position found: " + player_position)
        except:
            print("Still looking.")
    for game in game_list:
        stats = game.find_elements(By.TAG_NAME, "td")
        if len(stats) == 0 or stats[2].text == "Age":
            print("Still working")
        else:
            player_date = stats[1].text
            # This only scrapes a new line if it's actually new.
            if player_date not in existing_dates:            
                player_age = stats[2].text
                player_team = stats[3].text
                if stats[4].text == '@':
                    player_home_away = 'away'
                else:
                    player_home_away = 'home'
                player_playoff_reg = playoff_reg
                if len(stats) == 8:
                    player_status = stats[7].text
                    player_minutes = 0
                    player_FG = 0
                    player_FGA = 0
                    player_threeP = 0
                    player_threePA = 0
                    player_FT = 0
                    player_FTA = 0
                    player_ORB = 0
                    player_DRB = 0
                    player_AST = 0
                    player_STL = 0
                    player_BLK = 0
                    player_TOV = 0
                    player_PF = 0
                    player_plus_minus = 0
                else:
                    player_status = 'active'
                    player_minutes = stats[8].text
                    player_FG = stats[9].text
                    player_FGA = stats[10].text
                    player_threeP = stats[12].text
                    player_threePA = stats[13].text
                    player_FT = stats[15].text
                    player_FTA = stats[16].text
                    player_ORB = stats[18].text
                    player_DRB = stats[19].text
                    player_AST = stats[21].text
                    player_STL = stats[22].text
                    player_BLK = stats[23].text
                    player_TOV = stats[24].text
                    player_PF = stats[25].text
                    try:
                        player_plus_minus = int(stats[28].text)
                    except:
                        player_plus_minus = 0
            
                print("Adding new line to DB for: " + player_date)
                db_entry = [player_name,player_position,player_age,player_date,player_team, 
                            player_home_away, player_playoff_reg, player_status,player_minutes,
                            player_FG,player_FGA,player_threeP,player_threePA,player_FT,
                            player_FTA,player_ORB,player_DRB,player_AST,player_STL,player_BLK,
                            player_TOV,player_PF,player_plus_minus]
                player_cursor.execute("""INSERT INTO players(      
                                      name, position, age, date, team, home_away, playoff_reg,
                                      status, minutes, FG, FGA, threeP, threePA, FT, FTA, ORB,
                                      DRB, AST, STL, BLK, TOV, PF, plus_minus) VALUES (?, ?, 
                                      ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, 
                                      ?, ?)""", db_entry)
    player_connection.commit()


In [11]:
# This is the rowid collected from the db query above. It tells us the first new game entered into the db today.
start_entry = [int(games[0][0])]

# This is creating a list of unique player names that have been added in the most recent batch. 
schedule_cursor.execute("SELECT rowid, * FROM games WHERE rowid >= (?)", start_entry)
all_players = schedule_cursor.fetchall()
combined_player_list = []
for entry in all_players:
    for i in range(30):
        if entry[i+12] not in combined_player_list and len(entry[i+12]) > 5:
            combined_player_list.append(entry[i+12])

In [12]:
for player in tqdm(combined_player_list):
    # !!! This navigates to the current season's page. It will need to be updated each season. !!!
    # !!! It will also need to be changed during the playoffs. !!!
    driver.get("https://www.sports-website.com" + str(player.split(".htm")[0]) + "/gamelog/2023")
    time.sleep(1)

    # This then bounces to the game_scanner function.
    try:
        regular_games = driver.find_element(By.ID, "all_pgl_basic").find_elements(By.TAG_NAME, "tr")
        game_scanner(regular_games, player, 'regular')
    except:
        print("No regular games.")

    try:
        playoff_games = driver.find_element(By.ID, "all_pgl_basic_playoffs").find_elements(By.TAG_NAME, "tr")
        game_scanner(playoff_games, player, 'playoffs')
    except:
        print("No playoff games.")

  0%|                                                                                                                                            | 0/124 [00:00<?, ?it/s]

Player position found: Power Forward
Still working
Still working
Still working
Still working
Still working
Adding new line to DB for: 2023-04-06


  1%|█                                                                                                                                 | 1/124 [00:50<1:42:38, 50.07s/it]

No playoff games.
Player position found: Power Forward
Still working
Still working
Still working
Still working
Still working


  2%|██                                                                                                                                | 2/124 [01:57<2:02:21, 60.17s/it]

...

Adding new line to DB for: 2023-04-04
Adding new line to DB for: 2023-04-06
No playoff games.
Player position found: Shooting Guard
Still working
Still working
Still working
Still working


 99%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▉ | 123/124 [28:13<00:10, 10.80s/it]

Adding new line to DB for: 2023-04-06
No playoff games.
Player position found: Point Guard
Still working
Still working
Still working


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 124/124 [28:21<00:00, 13.72s/it]

Adding new line to DB for: 2023-04-06
No playoff games.


In [13]:
# This is updating the coach page with a W/L
for game in all_players:
    visitor_coach = game[7]
    home_coach = game[8]
    year = game[11]
    month = game[10]
    day = game[9]
    visitor_points = game[3]
    home_points = game[5]
    visitor_name = game[2]
    home_name = game[4]
    if visitor_points < home_points:
        home_WL = "W"
        visitor_WL = "L"
    elif visitor_points > home_points:
        home_WL = "L"
        visitor_WL = "W"
    else:
        home_WL = "T"
        visitor_WL = "T"       
    visitor_entry = [visitor_coach, visitor_name, year, month, day, visitor_WL]
    player_cursor.execute("""INSERT INTO coaches(      
                          coach_name, team_name, year, month, day, win_loss) VALUES (?, ?, 
                          ?, ?, ?, ?)""", visitor_entry)
    player_connection.commit()
    home_entry = [home_coach, home_name, year, month, day, home_WL]
    player_cursor.execute("""INSERT INTO coaches(      
                          coach_name, team_name, year, month, day, win_loss) VALUES (?, ?, 
                          ?, ?, ?, ?)""", home_entry)
    player_connection.commit()
    print(visitor_entry)
    print(home_entry)

['J.B. Bickerstaff', 'Cleveland Cavaliers', 2023, 4, 6, 'W']
['Jamahl Mosley', 'Orlando Magic', 2023, 4, 6, 'L']
['Erik Spoelstra', 'Miami Heat', 2023, 4, 6, 'W']
['Doc Rivers', 'Philadelphia 76ers', 2023, 4, 6, 'L']
['Chauncey Billups', 'Portland Trail Blazers', 2023, 4, 6, 'L']
['Gregg Popovich', 'San Antonio Spurs', 2023, 4, 6, 'W']
['Mark Daigneault', 'Oklahoma City Thunder', 2023, 4, 6, 'W']
['Will Hardy', 'Utah Jazz', 2023, 4, 6, 'L']
['Michael Malone', 'Denver Nuggets', 2023, 4, 6, 'L']
['Monty Williams', 'Phoenix Suns', 2023, 4, 6, 'W']


In [14]:
# This checks for sneaky missing coach data.

player_cursor.execute("SELECT rowid, team_name FROM coaches WHERE coach_name IS NULL")
missing_coaches = player_cursor.fetchall()

for coach in missing_coaches:
    team = [coach[1]]
    schedule_cursor.execute("SELECT visitor_coach FROM games WHERE visitor_name = (?) ORDER BY rowid DESC LIMIT 1", team)
    updated_coach = schedule_cursor.fetchall()
    data_entry = [updated_coach[0][0], coach[0]]
    print(data_entry)
    player_cursor.execute("UPDATE coaches SET coach_name = (?) WHERE rowid = (?)", data_entry)
    player_connection.commit()

In [15]:
# This is updating the day, month, year colums in the players/players DB.
player_cursor.execute("SELECT rowid, date, day, month, year FROM players WHERE day IS NULL")
empty_dates = player_cursor.fetchall()
for date in empty_dates:
    text_date = date[1].split('-')
    year = int(text_date[0])
    month = int(text_date[1])
    day = int(text_date[2])
    date_entry = [year, month, day, date[0]]
    player_cursor.execute("UPDATE players SET (year, month, day) = (?, ?, ?) WHERE rowid = (?)", date_entry)
    player_connection.commit()

In [16]:
# This needs to check for new players and add them to the dictionary:
url = "https://www.sports-website.com"
player_cursor.execute("SELECT DISTINCT link FROM dictionary")
old_links = []
batch = player_cursor.fetchall()
for link in batch:
    old_links.append(link[0])

player_cursor.execute("SELECT DISTINCT name FROM players")
new_links = []
batch = player_cursor.fetchall()
for link in batch:
    if link[0] not in old_links:
        new_links.append(link[0])
    
for i in new_links:
    link = url + i
    driver.get(link)
    time.sleep(3)
    spelling = driver.find_element(By.ID, 'meta').find_element(By.TAG_NAME, 'span').text
    data = [spelling, i]
    print(data)
    player_cursor.execute("""INSERT INTO dictionary(      
                      spelling, link) VALUES (?, ?)""", data)
    player_connection.commit()


['Justin Minaya', '/players/m/minayju01.html']


In [5]:
# This looks at each player entry and adds an injury status indicator. 
# 1 = healthy, 0 = scratch, .9 = probable, .5 = questionable, and .1 = doubtful. 
# If status is not active, probabilities are 25% scratch, 45% doubtful, 5% probable, and 25% questionable.
# If status is active but last game was not active, probabilities are 25% healthy, 25% questionable, 45% probable, 5% doubtful.

# # First we create a list of unique names from the player database.
player_cursor.execute("SELECT DISTINCT name FROM players WHERE injury IS NULL")
player_names = player_cursor.fetchall()

for player_name in player_names:
    name = player_name[0]
    print(name)
    player_cursor.execute("SELECT status, year, month, day, rowid, injury FROM players WHERE name = (?) ORDER BY year ASC, month ASC, day ASC", [name])
    status_list = player_cursor.fetchall()    
    last_injury = 'active'
    # This sets the player's injury status to 1 if it's their first game.
    table_input = [1, status_list[0][4]]
    if status_list[0][5] is None:
        player_cursor.execute("UPDATE players SET injury = (?) WHERE rowid = (?)", table_input)
        player_connection.commit()

    # This set's the player's injury status for all following games.
    i=0
    while i < len(status_list)-1:
        i += 1 # Why is this immediately incrementing by 1?
        # This sets the injury score if the player was not active.
        if status_list[i][0] != 'active' or status_list[i][0] != 'Did Not Play':
            outcome = random.random()
            if outcome < 0.25:
                this_injury = 0.0
            elif outcome < 0.7:
                this_injury = 0.1
            elif outcome < 0.95:
                this_injury = 0.5
            else:
                this_injury = 0.9
        # This sets the injury score if the player is active, but wasn't in the last game (or if this is the first game record).
        elif last_injury != 'active' or last_injury != 'Did Not Play':
            outcome = random.random()
            if outcome < 0.05:
                this_injury = 0.1
            elif outcome < 0.3:
                this_injury = 0.5
            elif outcome < 0.75:
                this_injury = 0.9
            else:
                this_injury = 1.0
        else:
            this_injury = 1.0

        last_injury = status_list[i][0]
        table_input = [this_injury, status_list[i][4]]
        if status_list[i][5] is None:
            player_cursor.execute("UPDATE players SET injury = (?) WHERE rowid = (?)", table_input)
            player_connection.commit()

/players/j/jamesle01.html
/players/n/nowitdi01.html
/players/r/roybr01.html
/players/a/artesro01.html
/players/j/johnsjo02.html
/players/b/boshch01.html
/players/p/pietrmi01.html
/players/a/allenra02.html
/players/w/wadedw01.html
/players/d/duranke01.html
/players/b/bonnema01.html
/players/b/breweco01.html
/players/m/mayooj01.html
/players/k/kamanch01.html
/players/e/ellismo01.html
/players/a/anthoca01.html
/players/s/salmojo01.html
/players/a/aldrila01.html
/players/w/wallage01.html
/players/i/iguodan01.html
/players/h/horfoal01.html
/players/p/piercpa01.html
/players/p/paulch01.html
/players/l/leeco01.html
/players/g/gayru01.html
/players/g/grangda01.html
/players/p/princta01.html
/players/k/kirilan01.html
/players/n/nashst01.html
/players/o/odomla01.html
/players/f/foyera01.html
/players/g/gallida01.html
/players/a/arizatr01.html
/players/j/jennibr01.html
/players/p/parketo01.html
/players/t/turkohe01.html
/players/g/greenje02.html
/players/m/martike01.html
/players/s/stoudam01.html

/players/b/bealbr01.html
/players/s/singlky01.html
/players/t/tinslja01.html
/players/t/tayloje03.html
/players/n/nealga01.html
/players/k/kanteen01.html
/players/l/lillada01.html
/players/s/shvedal01.html
/players/d/daniema01.html
/players/p/pondequ01.html
/players/f/farieke01.html
/players/v/vucevni01.html
/players/e/evansre01.html
/players/g/greenwi01.html
/players/z/zellety01.html
/players/c/clarkea01.html
/players/k/kiddgmi01.html
/players/k/korveky01.html
/players/b/butleji01.html
/players/t/thomptr01.html
/players/t/templga01.html
/players/a/allenla01.html
/players/n/nichoan01.html
/players/h/harklma01.html
/players/e/ellinwa01.html
/players/r/roberbr01.html
/players/c/crowdja01.html
/players/m/morrima02.html
/players/t/tuckepj01.html
/players/s/stiemgr01.html
/players/p/prigipa01.html
/players/a/anderja01.html
/players/m/mcrobjo01.html
/players/v/valanjo01.html
/players/d/decolna01.html
/players/p/pattepa01.html
/players/m/martica01.html
/players/b/bartowi01.html
/players/h/haw

/players/n/nunnke01.html
/players/l/laymaja01.html
/players/k/klebima01.html
/players/m/mannte01.html
/players/p/poeltja01.html
/players/h/hachiru01.html
/players/m/mykhasv01.html
/players/b/browntr01.html
/players/m/mahinia01.html
/players/c/culveja01.html
/players/n/napiesh01.html
/players/p/paytoga02.html
/players/w/willijo04.html
/players/j/jonesde02.html
/players/f/fultzma01.html
/players/d/doumbse01.html
/players/g/garlada01.html
/players/h/herroty01.html
/players/w/woodch01.html
/players/m/mclaujo01.html
/players/p/poolejo01.html
/players/r/reddica01.html
/players/p/porteke02.html
/players/w/whiteco01.html
/players/w/willizi01.html
/players/b/bacondw01.html
/players/j/johnske04.html
/players/c/clarkbr01.html
/players/j/joeis01.html
/players/t/tateja01.html
/players/v/vandeja01.html
/players/c/clarkga01.html
/players/t/trentga02.html
/players/m/maledth01.html
/players/a/anthoco01.html
/players/b/bazleda01.html
/players/k/korkmfu01.html
/players/b/ballla01.html
/players/d/diallha0

/players/r/ratlith01.html
/players/s/singlja01.html
/players/t/tayloje02.html
/players/e/elyme01.html
/players/c/childjo01.html
/players/c/collini01.html
/players/m/mcguido01.html
/players/j/jeffeot01.html
/players/e/ebankde01.html
/players/h/harrejo01.html
/players/t/thompmy02.html
/players/b/battito01.html
/players/p/pittmde01.html
/players/w/willijo03.html
/players/m/masonro01.html
/players/s/smithgr02.html
/players/e/ezelife01.html
/players/h/hamiljo02.html
/players/m/mekelga01.html
/players/t/teaguma01.html
/players/a/anticpe01.html
/players/m/moultar01.html
/players/p/papanko01.html
/players/f/freeljo01.html
/players/m/mcgarmi01.html
/players/o/onuakar01.html
/players/c/cunnija01.html
/players/e/ennisty01.html
/players/j/johnsch04.html
/players/o/obryajo01.html
/players/m/mcculch01.html
/players/l/liggide01.html
/players/m/mcclesh01.html
/players/r/reedwi02.html
/players/w/whiteok01.html
/players/s/swanica01.html
/players/w/wilsoja02.html
/players/r/richama01.html
/players/d/davi

/players/d/davisjo02.html
/players/f/fortsco01.html
/players/s/smithje01.html
/players/e/engliki01.html
/players/w/waynsma01.html
/players/h/hansbbe01.html
/players/m/murpher01.html
/players/b/babbch01.html
/players/c/cherrwi01.html
/players/d/datomlu01.html
/players/m/mcneaje01.html
/players/p/pleisti01.html
/players/h/harpeju01.html
/players/f/feldeka01.html
/players/z/zimmest01.html
/players/c/chrisra01.html
/players/l/longsh01.html
/players/q/quartti01.html
/players/u/uthofja01.html
/players/j/jacksde01.html
/players/m/magetjo01.html
/players/q/qizh01.html
/players/d/doylemi01.html
/players/p/peteral01.html
/players/w/whitean01.html
/players/c/cooleja01.html
/players/t/terreja01.html
/players/s/silvach01.html
/players/p/pelleno01.html
/players/m/mikaer01.html
/players/n/norveza01.html
/players/m/mokokad01.html
/players/t/terryty01.html
/players/m/mcderse01.html
/players/j/jonesma05.html
/players/m/maneka01.html
/players/f/frankro01.html
/players/m/martije02.html
/players/o/oliveca0

/players/s/sochaje01.html
/players/w/weslebl01.html
/players/b/branhma01.html
/players/b/braunch01.html
/players/w/watsope01.html
/players/a/agbajoc01.html
/players/f/fontesi01.html
/players/h/hardyja02.html
/players/w/walkeja01.html
/players/m/mitchda01.html
/players/m/monekch01.html
/players/g/greenaj01.html
/players/b/beaucma01.html
/players/k/keelstr01.html
/players/j/juzanjo01.html
/players/w/whiteja03.html
/players/h/halljo02.html
/players/d/diabamo01.html
/players/c/cainja01.html
/players/b/barlodo01.html
/players/b/butlejo01.html
/players/w/washity02.html
/players/l/lawsoaj01.html
/players/r/robinor01.html
/players/e/elliske01.html
/players/c/champju02.html
/players/d/daysda01.html
/players/b/badjiib01.html
/players/r/rhodeja01.html
/players/l/liddeej01.html
/players/w/willido02.html
/players/b/bouyeja01.html
/players/u/umudest01.html
/players/j/jacksqu01.html
/players/q/quinole01.html
/players/c/cooksxa01.html
/players/i/inglejo01.html
